#HW Wk 4. Data Cleaning
## Liam McKenzie

#Load in data and imports

In [1]:
import pandas as pd
import numpy as np
url = ("https://raw.githubusercontent.com/IndraniMandal/CSC310-S20/master/messy_covid19_southamerica%20-%20covid19_southamerica.csv")


In [2]:
c19df = pd.read_csv(url)
c19df.head()

,ObservationDate,Country_Region,Province_State,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,NaN,0.0,0,0,0.0
1,2020-01-23,Brazil,NaN,0.0,0,0,0.0
2,2020-01-23,Colombia,NaN,0.0,0,0,0.0
3,2020-02-26,Brazil,NaN,1.0,0,0,1.0
4,2020-02-26,Brazil,NaN,1.0,0,0,1.0


#Question one:
###Which column has the maximum missing value? What % value is missing in that column? Should you keep this column or drop it? Why?


In [3]:
c19df.isnull().sum()

,0
ObservationDate,0
Country_Region,0
Province_State,5989
Confirmed,1
Deaths,3
Recovered,2
Active,26


In [4]:
maxMissPerc = c19df.isnull().sum().max()/len(c19df)*100
maxMissPerc

25.870410367170628

Province_State has the highest missing value with ≈ 25.87% of the data missing. Based on the scope of the data set, it should be safe to drop this column.

In [5]:
c19df = c19df.drop('Province_State', axis=1)
c19df.head()

,ObservationDate,Country_Region,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,0.0,0,0,0.0
1,2020-01-23,Brazil,0.0,0,0,0.0
2,2020-01-23,Colombia,0.0,0,0,0.0
3,2020-02-26,Brazil,1.0,0,0,1.0
4,2020-02-26,Brazil,1.0,0,0,1.0


In [6]:
c19df.isnull().sum()

,0
ObservationDate,0
Country_Region,0
Confirmed,1
Deaths,3
Recovered,2
Active,26


#Question 2:
###There are some columns with only a few data points missing. Should you drop these columns too? Why or why not? How should you handle these specific data points(particular row)?

Now that Province_State is dropped, only a few null values exist. There is no reason to drop these values as they only have a small amount of NaNs and the columns provide useful information. Instead, they can be handled via imputation techniques like imputation via mean/median values.

Use imputation by mean value on 'Confirmed' because it is numerical data

In [7]:
c19df['Confirmed'].fillna(c19df['Confirmed'].mean(), inplace=True)


/tmp/ipython-input-3317063441.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  c19df['Confirmed'].fillna(c19df['Confirmed'].mean(), inplace=True)


In [8]:
c19df.Confirmed[c19df.Confirmed.isnull()]

,Confirmed


Use imputation by median on 'Deaths' because it holds categorical data.

In [9]:
c19df['Deaths'].fillna(0, inplace=True)

/tmp/ipython-input-2375528075.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  c19df['Deaths'].fillna(0, inplace=True)


In [10]:
c19df.Deaths[c19df.Deaths.isnull()]

,Deaths


Check data type for 'Recovered' for best strategy for imputation.

In [11]:
c19df['Recovered']

,Recovered
0,0
1,0
2,0
3,0
4,0
...,...
23145,64
23146,4667
23147,2275
23148,1755


Because 'Recovered' is integers stored as dtype object, imputation by kNN is necessary. First, the data must be converted to type numeric

In [12]:
c19df['Recovered'] = pd.to_numeric(c19df['Recovered'], errors='coerce')

Group 'Recovered' and 'Active' into one value 'v'

In [13]:
v = c19df.loc[: ,'Recovered':'Active']

In [14]:
v

,Recovered,Active
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
23145,64.0,0.0
23146,4667.0,67.0
23147,2275.0,2039.0
23148,1755.0,208.0


The simple value imputer needs to be imported in order to perform kNN imputation on the data set for 'Recovered'

In [15]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
clean_df = imputer.fit_transform(v)

In [16]:
#check imputation
clean_df

array([[    0.,     0.],
       [    0.,     0.],
       [    0.,     0.],
       ...,
       [ 2275.,  2039.],
       [ 1755.,   208.],
       [63346.,  9568.]])

The "new" data is not a part of the original data frame so it needs to be merged back in.

In [17]:
clean_df = pd.DataFrame(clean_df, columns= ['Recovered', 'Active'])
clean_df.head()

,Recovered,Active
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,1.0
4,0.0,1.0


In [18]:
c19df.loc[ : ,'ObservationDate': 'Deaths']

,ObservationDate,Country_Region,Confirmed,Deaths
0,2020-01-23,Mexico,0.0,0
1,2020-01-23,Brazil,0.0,0
2,2020-01-23,Colombia,0.0,0
3,2020-02-26,Brazil,1.0,0
4,2020-02-26,Brazil,1.0,0
...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,64.0,0
23146,2020-09-28,Suriname,4836.0,102
23147,2020-09-28,Trinidad and Tobago,4386.0,72
23148,2020-09-28,Uruguay,2010.0,47


Now that our kNN imputation is a dataframe we can concatinate it to the remaining dataset

In [19]:
transformed_df = pd.concat([c19df.loc[ : ,'ObservationDate': 'Deaths'],clean_df ], axis=1)
transformed_df

,ObservationDate,Country_Region,Confirmed,Deaths,Recovered,Active
0,2020-01-23,Mexico,0.0,0,0.0,0.0
1,2020-01-23,Brazil,0.0,0,0.0,0.0
2,2020-01-23,Colombia,0.0,0,0.0,0.0
3,2020-02-26,Brazil,1.0,0,0.0,1.0
4,2020-02-26,Brazil,1.0,0,0.0,1.0
...,...,...,...,...,...,...
23145,2020-09-28,Saint Vincent and the Grenadines,64.0,0,64.0,0.0
23146,2020-09-28,Suriname,4836.0,102,4667.0,67.0
23147,2020-09-28,Trinidad and Tobago,4386.0,72,2275.0,2039.0
23148,2020-09-28,Uruguay,2010.0,47,1755.0,208.0


#Question 3
### There are some data that has incorrect information like instead of a number they might have strings. How are you handling these data points(rows)?

First, check for remaining missing values or incorrect information in the dataset using our new df.

In [20]:
transformed_df.isnull().sum()

,0
ObservationDate,0
Country_Region,0
Confirmed,0
Deaths,0
Recovered,0
Active,0


In [21]:
c19df.isnull().sum()

,0
ObservationDate,0
Country_Region,0
Confirmed,0
Deaths,0
Recovered,5
Active,26


Since no missing values exist anymore, imputation techniques performed on the data set worked as intended.

#Question 4
### Display top ten countries with most number of confirmed cases

Group countries by confirmed cases

In [22]:
groupDF = transformed_df.groupby('Country_Region')['Confirmed'].max()

In [23]:
groupDF

,Confirmed
Country_Region,
Antigua and Barbuda,101.0
Argentina,723132.0
Bahamas,3838.0
Barbados,190.0
Belize,1891.0
Bolivia,134223.0
Brazil,973142.0
Chile,285939.0
Colombia,265576.0


Sort the grouped data by confirmed cases

In [24]:
sortedDF = groupDF.sort_values(ascending=False)

Display top 10 countries

In [25]:
sortedDF.head(10)

,Confirmed
Country_Region,
Brazil,973142.0
Argentina,723132.0
Peru,367068.0
Chile,285939.0
Colombia,265576.0
Ecuador,134965.0
Bolivia,134223.0
Mexico,124314.0
Dominican Republic,111666.0


Export Cleaned dataset

In [26]:
from google.colab import files

transformed_df.to_csv('cleaned_data.csv', index=False)
files.download('cleaned_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>